# Assignment 1

In this assignment, you'll be working with messy medical data and using regex to extract relevant infromation from the data. 

Each line of the `dates.txt` file corresponds to a medical note. Each note has a date that needs to be extracted, but each date is encoded in one of many formats.

The goal of this assignment is to correctly identify all of the different date variants encoded in this dataset and to properly normalize and sort the dates. 

Here is a list of some of the variants you might encounter in this dataset:
* 04/20/2009; 04/20/09; 4/20/09; 4/3/09
* Mar-20-2009; Mar 20, 2009; March 20, 2009;  Mar. 20, 2009; Mar 20 2009;
* 20 Mar 2009; 20 March 2009; 20 Mar. 2009; 20 March, 2009
* Mar 20th, 2009; Mar 21st, 2009; Mar 22nd, 2009
* Feb 2009; Sep 2009; Oct 2010
* 6/2008; 12/2009
* 2009; 2010

Once you have extracted these date patterns from the text, the next step is to sort them in ascending chronological order accoring to the following rules:
* Assume all dates in xx/xx/xx format are mm/dd/yy
* Assume all dates where year is encoded in only two digits are years from the 1900's (e.g. 1/5/89 is January 5th, 1989)
* If the day is missing (e.g. 9/2009), assume it is the first day of the month (e.g. September 1, 2009).
* If the month is missing (e.g. 2010), assume it is the first of January of that year (e.g. January 1, 2010).
* Watch out for potential typos as this is a raw, real-life derived dataset.

With these rules in mind, find the correct date in each note and return a pandas Series in chronological order of the original Series' indices. **This Series should be sorted by a tie-break sort in the format of ("extracted date", "original row number").**

For example if the original series was this:

    0    1999
    1    2010
    2    1978
    3    2015
    4    1985

Your function should return this:

    0    2
    1    4
    2    0
    3    1
    4    3

Your score will be calculated using [Kendall's tau](https://en.wikipedia.org/wiki/Kendall_rank_correlation_coefficient), a correlation measure for ordinal data.

*This function should return a Series of length 500 and dtype int.*

In [1]:
import pandas as pd

doc = []
with open('assets/dates.txt') as file:
    for line in file:
        doc.append(line)

df = pd.Series(doc)
df.head(10)

0         03/25/93 Total time of visit (in minutes):\n
1                       6/18/85 Primary Care Doctor:\n
2    sshe plans to move as of 7/8/71 In-Home Servic...
3                7 on 9/27/75 Audit C Score Current:\n
4    2/6/96 sleep studyPain Treatment Pain Level (N...
5                    .Per 7/06/79 Movement D/O note:\n
6    4, 5/18/78 Patient's thoughts about current su...
7    10/24/89 CPT Code: 90801 - Psychiatric Diagnos...
8                         3/7/86 SOS-10 Total Score:\n
9             (4/10/71)Score-1Audit C Score Current:\n
dtype: object

In [2]:
import re

def date_sorter():

    pattern = (
        # 04/20/2009; 04/20/09; 4/20/09; 4/3/09
        r'(?P<month>\d{1,2})[\/-](?P<day>\d{1,2})[\/-](?P<year>\d{2,4})|'
        # Mar-20-2009; Mar 20, 2009; March 20, 2009; Mar. 20, 2009; Mar 20 2009;
        # 20 Mar 2009; 20 March 2009; 20 Mar. 2009; 20 March, 2009
        # Mar 20th, 2009; Mar 21st, 2009; Mar 22nd, 2009
        # Feb 2009; Sep 2009; Oct 2010
        r'(?P<day_alt>\d{1,2})?\s?(?P<month_alt>Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-z]*[.\s,-]?\s?(?P<day_alt2>\d{1,2})?[a-z]*[.,\s-]?\s?(?P<year_alt>\d{2,4})|'
        # 6/2008; 12/2009
        # 2009; 2010
        r'(?P<month_slash>\d{1,2})?[\/-]?(?P<year_only>\d{4})'
    )

    # YOUR CODE HERE
    #raise NotImplementedError()

    # Extract date components
    order = df.astype(str).str.extract(pattern)

    # Fill missing values from alternative regex groups
    order['month'] = order['month'].fillna(order['month_alt']).fillna(order['month_slash'])
    order['day'] = order['day'].fillna(order['day_alt']).fillna(order['day_alt2'])
    order['year'] = order['year'].fillna(order['year_alt']).fillna(order['year_only'])

    # Convert month names to numbers
    month_map = {
        'January': '01', 'February': '02', 'March': '03', 'April': '04',
        'May': '05', 'June': '06', 'July': '07', 'August': '08',
        'September': '09', 'October': '10', 'November': '11', 'December': '12',
        'Jan': '01', 'Feb': '02', 'Mar': '03', 'Apr': '04', 'May': '05', 'Jun': '06',
        'Jul': '07', 'Aug': '08', 'Sep': '09', 'Oct': '10', 'Nov': '11', 'Dec': '12'
    }
    order['month'] = order['month'].replace(month_map)

    # Convert two-digit years to four-digit (assuming 1900s for values < 25)
    order['year'] = order['year'].astype(str).apply(lambda x: f'20{x}' if len(x) == 2 and int(x) < 25 else (f'19{x}' if len(x) == 2 else x))

    # Fill NaN values with empty strings
    order['month'] = order['month'].fillna('1')
    order['day'] = order['day'].fillna('1')
    
    # Concat Columns
    order['combined'] = order[['month', 'day', 'year']].apply(lambda row: '-'.join(row.values.astype(str)), axis=1)
    order = order[['combined']]

    return order

In [3]:
order = date_sorter()

formatted_dates = order.apply(lambda row: ' '.join(row.dropna()), axis=1)
print("\n".join(formatted_dates))

03-25-1993
6-18-1985
7-8-1971
9-27-1975
2-6-1996
7-06-1979
5-18-1978
10-24-1989
3-7-1986
4-10-1971
5-11-1985
4-09-1975
8-01-1998
1-26-1972
5-24-1990
1-25-2011
4-12-1982
10-13-1976
4-24-1998
5-21-1977
7-21-1998
10-21-1979
3-03-1990
2-11-1976
07-25-1984
4-13-1982
9-22-1989
9-02-1976
9-12-1971
10-24-1986
03-31-1985
7-20-1972
4-12-1987
06-20-1991
5-12-2012
3-15-1983
2-14-1973
5-24-1988
7-27-1986
1-14-1981
7-29-1975
6-24-1987
8-14-1994
4-13-2002
8-16-1982
2-15-1998
7-15-1991
06-12-1994
9-17-1984
2-28-1975
11-22-1975
5-24-1991
6-13-1992
7-11-1971
12-26-1986
10-11-1987
3-14-1995
12-01-1973
12-5-2010
08-20-1982
7-24-1995
8-06-1983
02-22-1992
6-28-1987
07-29-1994
08-11-1978
10-29-1991
7-6-1991
1-21-1987
11-3-1985
7-04-1982
4-13-1989
7-11-1977
4-12-1974
09-19-1981
9-6-1979
12-5-1987
01-05-1999
4-22-1980
10-04-1998
6-29-1981
8-04-1978
7-07-1974
09-14-2000
5-18-1971
8-09-1981
6-05-1993
8-9-1997
12-8-1982
8-26-1989
10-13-1995
4-19-1991
04-08-2004
9-20-1976
12-08-1990
4-11-1974
7-18-1986
3-31-1991
5

In [4]:
order

,combined
0,03-25-1993
1,6-18-1985
2,7-8-1971
3,9-27-1975
4,2-6-1996
...,...
495,1-1-1979
496,1-1-2006
497,1-1-2008
498,1-1-2005
